## 펀드별 핵심 키워드 추출 및 동시출현단어 분석
키워드를 기반하여 번액펀드를 추천할 수 있도록 펀드별 Feature를 잘 뽑아낼 수 있는 키워드 추출하고자 합니다. Mecab 라이브러리의 형태소 분석기와 사용자 사전 기능을 사용하여 키워드를 뽑아 내고 TF-IDF를 이용하여 펀드를 설명하는 텍스트의 핵심 키워드를 추출하였습니다. 
또한 단어별로 동시출현빈도를 구해 키워드간 유사도를 구하고 이를 네트워크 그래프로 시각화 하기 위한 좌표값을 구하여 시각화에 사용했습니다.

[목차]

0. Mecab 라이브러리 설치
1. 데이터 불러오기 및 전처리 
2. 사용자 사전 구축
3. 텍스트 정제 및 형태소 분석
4. TF-IDF 키워드 추출
5. 동시출현단어 분석 및 시각화 데이터 완성

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 0. Mecab 라이브러리 설치



In [2]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4 MB 49.9 MB/s 
     |████████████████████████████████| 448 kB 41.1 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-09-30 07:58:26--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff0

### 1. 데이터 불러오기 & 전처리

In [3]:
# 데이터 불러오기
txt_data = pd.read_csv("/content/drive/MyDrive/miraeasset/자산운용보고서_크롤링결과.csv", encoding ='cp949')
mali1 = pd.read_csv("/content/drive/MyDrive/miraeasset/mali_data1.csv")

In [4]:
# 텍스트 데이터 병합
fund_info = mali1[['fund_cd', 'fund_nm', 'fund_expln', 'stck_expln', 'finan_bond_expln', 'fund_diff_cd', 'fund_aset_cd', 'fund_stle_cd']].drop_duplicates()
fund_keyword = pd.merge(fund_info, txt_data, how = 'left', on = 'fund_cd')

# 기존 펀드상품설명서와 자산운용서 보고서에서 추출 텍스트 데이터 하나의 문서로 합치기
fund_keyword['text'] = fund_keyword[['fund_expln', 'stck_expln', 'finan_bond_expln', 'strategy', 'invest_point', 'progress_plan']].apply(' '.join, axis=1) 

### 2. 사용자 사전 구축

In [5]:
%cd /content/mecab-ko-dic-2.1.1-20180720
%ls
%ls user-dic 

/content/mecab-ko-dic-2.1.1-20180720
aclocal.m4      EF.csv       MAJ.csv      NorthKorea.csv     tools/
AUTHORS         EP.csv       Makefile     NP.csv             unk.def
autogen.sh*     ETM.csv      Makefile.am  NR.csv             unk.dic
ChangeLog       ETN.csv      Makefile.in  Person-actor.csv   user-dic/
char.bin        feature.def  matrix.bin   Person.csv         VA.csv
char.def        Foreign.csv  matrix.def   Place-address.csv  VCN.csv
clean*          Group.csv    missing*     Place.csv          VCP.csv
CoinedWord.csv  Hanja.csv    MM.csv       Place-station.csv  VV.csv
config.log      IC.csv       model.bin    pos-id.def         VX.csv
config.status*  Inflect.csv  model.def    Preanalysis.csv    Wikipedia.csv
configure*      INSTALL      NEWS         README             XPN.csv
configure.ac    install-sh*  NNBC.csv     rewrite.def        XR.csv
COPYING         J.csv        NNB.csv      right-id.def       XSA.csv
dicrc           left-id.def  NNG.csv      Symbol.csv         XS

In [6]:
# User dictionary의 고유어사전(nnp.csv) 파일 열기
with open("./user-dic/nnp.csv", 'r', encoding='utf-8') as f: 
  file_data = f.readlines()
file_data

['대우,,,,NNP,*,F,대우,*,*,*,*,*\n', '구글,,,,NNP,*,T,구글,*,*,*,*,*\n']

In [7]:
## 사용자 사전에 입력할 키워드

# 종목약명
stock_info = pd.read_csv("/content/drive/MyDrive/miraeasset/krx_종목기본정보.csv", encoding="cp949")
stock_nm_list = stock_info['한글 종목약명'].to_list()

# 키워드풀
keyword_pool = pd.read_csv("/content/drive/MyDrive/miraeasset/keyword_pool_전처리.csv")
pool_list = keyword_pool['theme'].to_list()

# 기타 단어
etc_list = ['이머징', '하이일드', '헬스케어', '친디아', '듀레이션', '수혜주', '성장주', '전략적', '이스트스프링', '크레딧', 
            'non-Agency', '커머디티', '스프레드', '블랙록', '테이킹', '신흥국가', '플랫폼', '고배당', '저배당', '국공채',
            '백신', '토탈리턴', '경우', '스마트베타', '신재생', '배당주', '맥스초이스']

# 고유어 리스트 생성
nnp_list = stock_nm_list + pool_list + etc_list            

In [8]:
!pip install jamo

# 종성 여부 판단 함수 - 받침 여부 반환
from jamo import h2j, j2hcj
def get_jongsung_TF(sample_text): 
  sample_text_list = list(sample_text) 
  last_word = sample_text_list[-1] 
  last_word_jamo_list = list(j2hcj(h2j(last_word))) 
  last_jamo = last_word_jamo_list[-1]
  
  jongsung_TF = "T"
   
  if last_jamo in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅘ', 'ㅚ', 'ㅙ', 'ㅝ', 'ㅞ', 'ㅢ', 'ㅐ,ㅔ', 'ㅟ', 'ㅖ', 'ㅒ']: 
    jongsung_TF = "F" 
   
  return jongsung_TF

In [9]:
# 고유어 리스트를 사용자 고유어 사전(nnp.csv)에 추가
for word in nnp_list:
  jongsung_TF = get_jongsung_TF(word)
  line = '{},,,0,NNP,*,{},{},*,*,*,*,*\n'.format(word, jongsung_TF, word) # 단어비용 0으로 설정하여 검색 우선순위 높임
  file_data.append(line)

with open("./user-dic/nnp.csv", 'w', encoding='utf-8') as f: 
  for line in file_data: f.write(line)

In [10]:
# 추가된 단어 확인 
with open("./user-dic/nnp.csv", 'r', encoding='utf-8') as f: 
  file_new = f.readlines() 

file_new[:10]

['대우,,,,NNP,*,F,대우,*,*,*,*,*\n',
 '구글,,,,NNP,*,T,구글,*,*,*,*,*\n',
 '마이크로컨텍솔,,,0,NNP,*,T,마이크로컨텍솔,*,*,*,*,*\n',
 '스카이이앤엠,,,0,NNP,*,T,스카이이앤엠,*,*,*,*,*\n',
 '포스코엠텍,,,0,NNP,*,T,포스코엠텍,*,*,*,*,*\n',
 'AJ네트웍스,,,0,NNP,*,F,AJ네트웍스,*,*,*,*,*\n',
 'AK홀딩스,,,0,NNP,*,F,AK홀딩스,*,*,*,*,*\n',
 'BGF리테일,,,0,NNP,*,T,BGF리테일,*,*,*,*,*\n',
 'BGF,,,0,NNP,*,T,BGF,*,*,*,*,*\n',
 'BNK금융지주,,,0,NNP,*,F,BNK금융지주,*,*,*,*,*\n']

In [11]:
# 추가한 단어들 사전에 입력
%ls tools
!bash ./tools/add-userdic.sh
!make install

add-userdic.sh*  convert_for_using_store.sh*  mecab-bestn.sh*
generating userdic...
nnp.csv
/content/mecab-ko-dic-2.1.1-20180720/tools/../model.def is not a binary model. reopen it as text mode...
reading /content/mecab-ko-dic-2.1.1-20180720/tools/../user-dic/nnp.csv ... 
done!
person.csv
/content/mecab-ko-dic-2.1.1-20180720/tools/../model.def is not a binary model. reopen it as text mode...
reading /content/mecab-ko-dic-2.1.1-20180720/tools/../user-dic/person.csv ... 
done!
place.csv
/content/mecab-ko-dic-2.1.1-20180720/tools/../model.def is not a binary model. reopen it as text mode...
reading /content/mecab-ko-dic-2.1.1-20180720/tools/../user-dic/place.csv ... 
done!
test -z "model.bin matrix.bin char.bin sys.dic unk.dic" || rm -f model.bin matrix.bin char.bin sys.dic unk.dic
/usr/local/libexec/mecab/mecab-dict-index -d . -o . -f UTF-8 -t UTF-8
reading ./unk.def ... 13
emitting double-array: 100% |###########################################| 
reading ./NNB.csv ... 140
reading ./XSA.

### 3.텍스트 정제 + 형태소분석

In [12]:
# 불용어 사전 불러오기
stopwords = pd. read_csv("/content/drive/MyDrive/miraeasset/불용어_사전.csv")
stopwords = stopwords['stopword'].to_list()
stopwords += ['분과', '평가손익', '대두', '억불', '보단','목표연도', '해당', '커머', '더티', '요소', '교지']

In [14]:
# Mecab 명사/영어 키워드 추출 
from konlpy.tag import Mecab
mecab = Mecab()

text_list = fund_keyword['text'].to_list()

# 토큰화 함수 생성 
def tokenizer(text_list):
  corpus = []

  for text in text_list:
    tag = mecab.pos(text) # 품사 태깅
    nouns = [s for s, t in tag if t in ['SL','NNG','NNP'] and s not in stopwords and len(s)>1]  # 명사/영단어이면서 불용어가 아닌 2자리이상 단어들 추출
    p = " ".join(nouns)
    corpus.append(p)

  return(corpus) 
corpus = tokenizer(text_list)

In [15]:
corpus[52]

'이머징 국공채 모기지 선진국 국공채 하이일드 격등 다양 안정 인컴 수익 추구 부동산 유동 전략 선진국 이머징 국공채 모기지 하이일드 다양 안정 인컴 수익 추구 포인트 미국 국공채 모기지 등급 회사채 이머징 달러 분산 포트폴리오 인컴 수익 추구 경과 향후 계획 인컴 기초 기간 상당 금리 상승 미국 듀레이션 신흥 듀레이션 익스포저 성과 저해 섹터 배분 선별 종목 선정 해분 상당 상쇄 성과 시현 스프레드 축소 등급 하이일드 회사채 캐리 종목 선정 non-Agency Agency MBS 익스포저 성과 기여 기대 인플레이션 상승 미국 물가 연동 국채 익스포저 성과 경과 향후 유동 유지 수익 기회 포착 미국 듀레이션 선호 인플레 대응 TIPS 익스포저 스프레드 전략 매력 리스크 조정 수익 제공 non Agency MBS 선호 Agency MBS 우량 CMBS 선호 회사채 은행 선호 운송 섹터 일부 선별 익스포저 통화 경우 변동성 감안 전술 수익 기회 포착 예정 향후 계획'

### 4. TF-IDF 키워드 추출

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict

# TF-IDF 단어 벡터화
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(corpus)

# word와 id 연결
word2id = defaultdict(lambda : 0)
word2id

for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

In [17]:
# 펀드별 상위 10개의 단어 추출
keyword = []
for i in range(0, len(corpus)) :
    sent = corpus[i]
    a = [(token, sp_matrix[i, word2id[token]]) for token in sent.split()]
    a = list(set(a))
    b = sorted(a, key=lambda score: score[1], reverse=True)
    c = b[0:10]
    d = []
    for i in range(0, len(c)):
        d.append(c[i][0])
    keyword.append(d)

In [18]:
fund_keyword['keyword'] = keyword
print(fund_keyword['keyword'])

0      [인프라, 코로나19, 아시아, 배당, 섹터, 종식, 미국, 재투자, 국가, 누적]
1            [단기, 콜론, 금리, 양도, 증서, 금융, 매수, 예금, 우려, 만기]
2           [기여, 실적, 증감, 국내, 가능, 업종, 반도체, 예정, 부정, 이뮨]
3        [고배당, 배당, 수익, 코로나19, 안정, 섹터, 종식, 미국, 자본, 증식]
4        [단기, 미국, 국채, 공사, 정책, 표시, 인플레이션, 크레딧, 경기, 은행]
                            ...                      
149         [상품, 파생, 국내, 장외, 수익, 지수, 대부분, 증권, 확보, 안정]
150         [중국, 기업, 산업, 대표, 규제, 플랫폼, 증시, 이슈, 이익, 주요]
151     [초이스, 맥스, 이스트스프링, 신탁, 증권, 집합, 보험, 현금, 안정, 상품]
152         [중국, 인도, 기업, 절반, 인구, 대표, 변화, 적응, 차지, 수혜주]
153     [초이스, 맥스, 이스트스프링, 신탁, 보험, 증권, 집합, 현금, 안정, 상품]
Name: keyword, Length: 154, dtype: object


In [19]:
from google.colab import files
fund_keyword.to_csv('fund_keyword.csv', index=False, encoding='cp949')
files.download('fund_keyword.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 5. 동시출현단어 분석 및 시각화 데이터 완성

- 단어의 동시 출현 빈도로 키워드 간 연관도 구하기

In [20]:
# 명사 추출 함수 
def tokenizer1(text_list):
  tag = mecab.pos(text_list)
  nouns = [s for s, t in tag if (t in ['SL','NNG','NNP']) and (s not in stopwords) and (len(s)>1)]
  return(nouns)

In [21]:
count = {} # 동시출현 빈도가 저장될 dict

for line in fund_keyword['text']:
    tokens = tokenizer1(line)
    stopped_tokens = [i for i in list(set(tokens))]
    stopped_tokens2 = [i for i in stopped_tokens if len(i) > 1]
    for i,a in enumerate(stopped_tokens2):
        for b in stopped_tokens2[i+1:]:
            if a>b:
                count[b,a] = count.get((b,a),0) + 1
            else:
                count[a,b] = count.get((a,b),0) + 1

In [22]:
# 두 단어의 동시출현 빈도 데이터프레임 
df = pd.DataFrame.from_dict(count, orient="index")
df = df.reset_index()
df.head()

,index,0
0,"(계획, 유동)",138
1,"(계획, 부양책)",65
2,"(계획, 국가)",34
3,"(MMF, 계획)",38
4,"(계획, 금리)",136


In [23]:
# 열별로 단어 구분
df1 = pd.DataFrame()
df1['word1'] = [df['index'][i][0] for i in range(0, len(df))]
df1['word2'] = [df['index'][i][1] for i in range(0, len(df))]
df1['freq'] = df.iloc[:,1]
data_set = df1[['word1', 'word2', 'freq']]
data_set.head()

,word1,word2,freq
0,계획,유동,138
1,계획,부양책,65
2,계획,국가,34
3,MMF,계획,38
4,계획,금리,136


-  네트워크 그래프 시각화 데이터 전처리

In [24]:
import networkx as nx
import operator
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

In [25]:
# word1에는 TF-IDF로 추출된 핵심 키워드만 입력
keyword = []
for i in range(0, len(fund_keyword.keyword)):
  keyword += fund_keyword['keyword'][i]
keyword = pd.DataFrame({'keyword':keyword})

In [26]:
keyword = keyword.groupby('keyword').size().reset_index()
keyword.columns = ['keyword', 'count']

In [27]:
keyword = keyword.sort_values(by="count", ascending=False).head(30).reset_index(drop = True)
keyword.head()

,keyword,count
0,코로나19,50
1,섹터,41
2,미국,40
3,종식,35
4,수익,29


In [28]:
# 네트워크 시각화 좌표 데이터프레임
xy_keyword = pd.DataFrame()


for i in range(0, len(keyword)):
  
  # 선택 단어만 필터링
  word = keyword.keyword[i]
  dataset = data_set[(data_set['word1']==word )| (data_set['word2']==word)] # 선택한 단어만 조회
  dataset = dataset.reset_index(drop = True)
  dataset = dataset.sort_values(by="freq", ascending=False).head(20).reset_index(drop = True)
  
  # 필터링된 데이터로 네트워크 그래프 생성
  if __name__ == '__main__':

    G_centrality = nx.Graph()
    
    for ind in range(0,len(dataset)):
      G_centrality.add_edge(dataset['word1'][ind], dataset['word2'][ind], weight = int(dataset['freq'][ind]))
        
    dgr = nx.degree_centrality(G_centrality)
    btw = nx.betweenness_centrality(G_centrality)
    cls = nx.closeness_centrality(G_centrality)
    egv = nx.eigenvector_centrality(G_centrality)
    pgr = nx.pagerank(G_centrality)
    
    sorted_dgr = sorted(dgr.items(), key = operator.itemgetter(1), reverse = True)
    sorted_btw = sorted(btw.items(), key = operator.itemgetter(1), reverse = True) 
    sorted_cls = sorted(cls.items(), key = operator.itemgetter(1), reverse = True) 
    sorted_egv = sorted(egv.items(), key = operator.itemgetter(1), reverse = True) 
    sorted_pgr = sorted(pgr.items(), key = operator.itemgetter(1), reverse = True)
    
    G = nx.Graph()
    
    for j in range(len(sorted_pgr)):
        G.add_node(sorted_pgr[j][0], nodesize = sorted_dgr[j][1])
    
    for ind in range(0,len(dataset)):
        G.add_weighted_edges_from([(dataset['word1'][ind], dataset['word2'][ind], int(dataset['freq'][ind]))])
      
  # node 좌표 추출
  position = nx.spring_layout(G)
  position_df = pd.DataFrame.from_dict(position, orient='index').reset_index()
  position_df.columns = ['word', 'X', 'Y']
  position_df['Relationship'] = position_df.word[0]+ '->' +position_df.word
  position_df = position_df.reset_index()
  position_df.columns = ['id', 'word', 'X', 'Y', 'Relationship']
  
  # 태블로 시각화를 위한 전처리
  a = position_df[['Relationship']]
  a['word'] = position_df.word[0]
  a['X'] = position_df.X[0]
  a['Y'] = position_df.Y[0]
  a = a.reset_index()
  a.columns = ['id', 'Relationship', 'word', 'X', 'Y']
  b = pd.concat([a, position_df]).reset_index(drop = True)
  b = b[b.id > 0].reset_index(drop = True)
  b['keyword'] = keyword.keyword[i]

  xy_keyword = pd.concat([xy_keyword,b])

  #print(i)

In [29]:
# 데이터 저장
xy_keyword.to_csv("/content/drive/MyDrive/miraeasset/대시보드용 데이터/network_data2.csv", index = False, encoding = 'utf-8-sig')
keyword.to_csv("/content/drive/MyDrive/miraeasset/대시보드용 데이터/network_data(keyword_cloud)2.csv", index = False, encoding = 'utf-8-sig')

In [30]:
# 펀드정보를 키워드별로 연결하는 데이터 생성
fund_keyword_list = []
fund_cd_list = []
fund_nm_list = []
fund_diff_cd_list = []
fund_expln_list = []
fund_aset_cd_list = []
fund_stle_cd_list = []
fund_expln_list = []

for i in range(0, len(fund_keyword.keyword)):
  for a in fund_keyword.keyword[i]:
    fund_keyword_list.append(a)
    fund_cd_list.append(fund_keyword.fund_cd[i])
    fund_nm_list.append(fund_keyword.fund_nm[i])
    fund_diff_cd_list.append(fund_keyword.fund_diff_cd[i])
    fund_expln_list.append(fund_keyword.fund_expln[i])
    fund_aset_cd_list.append(fund_keyword.fund_aset_cd[i])
    fund_stle_cd_list.append(fund_keyword.fund_stle_cd[i])

dat2 = pd.DataFrame({"fund_keyword":fund_keyword_list,
                     'fund_cd' : fund_cd_list,
                     'fund_nm' : fund_nm_list,
                     'fund_expln' : fund_expln_list,
                     'fund_diff_cd':fund_diff_cd_list,
                     'fund_aset_cd': fund_aset_cd_list,
                     'fund_stle_cd': fund_stle_cd_list})

dat2.head()

,fund_keyword,fund_cd,fund_nm,fund_expln,fund_diff_cd,fund_aset_cd,fund_stle_cd
0,인프라,N2C0,아시아인프라,자산의 60% 이상을 아시아 국가의 인프라관련 주식 및 아시아국가 인프라 주식을 주...,2,8,6
1,코로나19,N2C0,아시아인프라,자산의 60% 이상을 아시아 국가의 인프라관련 주식 및 아시아국가 인프라 주식을 주...,2,8,6
2,아시아,N2C0,아시아인프라,자산의 60% 이상을 아시아 국가의 인프라관련 주식 및 아시아국가 인프라 주식을 주...,2,8,6
3,배당,N2C0,아시아인프라,자산의 60% 이상을 아시아 국가의 인프라관련 주식 및 아시아국가 인프라 주식을 주...,2,8,6
4,섹터,N2C0,아시아인프라,자산의 60% 이상을 아시아 국가의 인프라관련 주식 및 아시아국가 인프라 주식을 주...,2,8,6


In [31]:
dat2.to_csv('/content/drive/MyDrive/miraeasset/대시보드용 데이터/network_data(match)2.csv', index=False, encoding='cp949')

본 저작물의 저작권은 Apache License v2.0을 따릅니다.